In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/data/processed/final_data.csv")


In [1]:
movies.head()
print("H")

NameError: name 'movies' is not defined

In [3]:
print("HOLA")

HOLA


In [4]:
import nltk

In [5]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [6]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
movies['tags'] = movies['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import joblib
cv = CountVectorizer(
    max_features=50000,
    stop_words="english"
)

# IMPORTANT: keep it SPARSE
X = cv.fit_transform(movies["tags"])

joblib.dump(X, "/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/models/movie_features.pkl")
joblib.dump(cv, "/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/models/modelscount_vectorizer.pkl")

print("✅ Features saved")
print("Shape:", X.shape)

In [ ]:
movies.to_csv("/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/data/processed/final_data.csv",index = False)

In [ ]:
import joblib

X_sparse = joblib.load(
    "/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/models/movie_features.pkl"
)
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(
    n_components=300,   # sweet spot (200–400)
    random_state=42
)

X_reduced = svd.fit_transform(X_sparse)
X = X_reduced.astype("float32")
import faiss

faiss.normalize_L2(X)
index = faiss.IndexFlatIP(X.shape[1])
index.add(X)
faiss.write_index(
    index,
    "/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/models/faiss.index"
)
joblib.dump(
    svd,
    "/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/models/svd.pkl"
)


In [ ]:
joblib.dump(
    X,
    "/Users/nidhishgupta/Desktop/Movie_recommendation_sysytem/models/movie_vectors_reduced.pkl"
)